# Mappe del 2

Her er all info og besvarelse fra andre del av prosjektet. Denne delen har fokusert på å ta i bruk dataene hentet ut og formatterte i første del, for så å visualisere det og lage en enkel prediksjonsmodell. I dette prosjektet er det fokus på nedbør og å kunne forutsi om det vil regne dagen etterpå eller ikke. Det gir en praktisk anvendelse av prosjektet som helhet og kan eksempelvis knyttes opp mot om man skal ta med seg paraply eller ikke.

## Oppgave 4

Første oppgave i andre del av prosjektet fokuserte på analysen av dataene. Her analyseres data og det beregnes gjennomsnitt, median og standardavvik. Dette steget ga et innsyn i dataene og dannet en forståelse som gjorde videre arbeid enklere.

## Oppgave 5

Analysen av dataene med de statistiske målene ble nyttige for å videre kunne visualisere dataene og se hvordan elementer som nedbør faktisk så ut over flere år. Her ble det i utgangspunktet plottet mye på grunn av et fokus på å forstå enda mer av dataene og hvordan de kunne tas i bruk i siste steg for prediksjon. Det ble mot slutten av prosjektet kuttet ned på mengden visualsering for å holde det ryddig og enkelt.

## Oppgave 6

Den mest krevende oppgaven om prediksjon er et forsøk på å forutsi nedbør en dag i forveien. Her benyttes to maskinlæringsmodeller, logistisk regresjon og Naive Byes, for å gi en indikasjon på om det vil regne eller ikke. Etter å ha målt nøyaktigheten av hver av modellene viste det seg at logistisk regresjon var mest nøyaktiig. Den ble derfor brukt i siste del av oppgaven som da forteller om det blir regn dagen etterpå eller ikke basert på maksimum- og minimumstemperaturen for dagen i dag.


## Generelt

### Håndtering og kartlegging av manglende værdata

For å sikre en systematisk tilnærming til manglende data har vi utviklet tre separate Python-moduler. Én modul kartlegger og eksporterer rå manglende datarader til egne CSV-filer, én modul inneholder funksjoner for visualisering av fraværende verdier (stolpe-, heatmap- og tidslinjegraf), og én modul tilbyr interpolerings¬metoder for å fylle igjen hull i tidsseriene. Denne oppdelingen gir tydelig ansvar og gjenbrukbarhet og gir et konkret grunnlag for videre analyse, dokumentasjon og kvalitetskontroll.

I en tilhørende Jupyter-notebook benytter vi stolpediagrammer for å gi rask oversikt over hvilke variabler (f.eks. max- og min-temperatur, vindhastighet, nedbør) og byer som har størst omfang av manglende målinger. Korrelasjons-heatmaps avdekker at max(air_temperature) og min(air_temperature) ofte mangler samtidig – trolig fordi sensor¬nedetid eller kalibrerings¬perioder påvirker begge temperatur¬registrene parallelt. For å visualisere trender og «lommer» av mangler over tid har vi videre utviklet en egendefinert tidslinje-graf, som erstatter Missingno-modulens matrise¬funksjon når datasettet spenner over 24 år og blir for uoversiktlig. Til slutt har vi laget en enkel Dash-app som viser antall mangler per variabel og by, og lar brukeren filtrere for én-til-én-sammenligninger uten behov for å laste inn nye statiske figurer.

Ut ifra de visualiseringen vi har gjort over har vi identifisert av vi hovedsakelig har to type manglende data. Sporadiske mangler og lengere perioder med manglende data. Dette gjør at vi har valgt en todelt interpoleringstilnærming. For små hull – definert som sammenhengende manglende perioder under tre dager – benyttes enkel lineær interpolering, som ivaretar kontinuiteten uten å introdusere kunstige sesongtrender. For større perioder, der både sesongvariasjon og langsiktige trender er viktige, har vi implementert sesongjustert interpolering ved hjelp av glidende vinduer og spline-funksjoner med sesongkomponent (seasonal spline). Denne hybridmetoden balanserer fleksibilitet og robusthet, og er valgt for å minimere systematisk bias i eksempelvis sommer- og vintermålinger.


### Hvordan FutureWarnings blir undertrykket

Vi har valgt å importere warnings for å undertrykke en spesifikk FutureWarning som oppstår når missingno.heatmap() benytter Seaborns interne formaterings‐argument (fmt) som i kommende Matplotlib‐utgaver kan fjernes eller endres. Det finnes ingen annen løsning på kort sikt fordi advarselen stammer fra en intern implementasjon i Seaborn/missingno som vi ikke kan endre direkte i vårt prosjekt. Kompatibiliteten mellom de aktuelle versjonene av Seaborn (0.12.x) og Matplotlib (3.6+) er foreløpig ikke fullstendig avklart upstream, og inntil en offisiell patch er utgitt, er undertrykkelse av akkurat denne advarselen den tryggeste måten å sikre ryddig logging og uforstyrret kjøring av våre notebooks.

Vi har sjekket flere kilder—inkludert Seaborn‐dokumentasjon, GitHub‐issues og diskusjonstråder på Stack Overflow—og konkludert med at det per i dag ikke finnes en permanent utgivelse som fjerner bruken av fmt i Seaborns matrix.py eller i missingno. Å oppgradere til Seaborn ≥ 0.13.x eller vente på Missingno ≥ 0.6.0 vil tidligst gi en løsning den dagen de utgir en patch som tar høyde for endringene i Matplotlibs håndtering av annotasjoner.

### Navigering av filstruktur

Vi har løst modulpakking og filnavigasjon ved konsekvent å benytte oss av relative filstier gjennom sys.path.append i stedet for å stole på os-metoder som henter absolutte stier. Dette gir en mer dynamisk kode struktur som bare forholder seg til det relative forholdet mellom python modul, data fil og/eller jupiter notebook. Hovedårsaken til å unngå absolutte stier i Jupyter-notebooks er at all output – inkludert filstier som logges eller printes – lagres direkte i .ipynb-filen. Med absolutte stier eksponeres hele katalogbanen til brukeren som kjører notebooken, inkludert maskinens brukernavn og den fulle prosjektstrukturen. Siden vårt GitHub-repositorium er offentlig, kan dette bryte med personvernprinsipper og avsløre sensitiv informasjon. Ved å bruke relative stier ser man derimot kun filens plassering i forhold til notebooken, noe som beskytter brukerdata


### Håndtering av .env-filen og .gitignore

I løpet av prosjektet valgte vi midlertidig å ikke ekskludere .env-filen, med hensikt å pushe den til GitHub slik at alle utviklere enkelt kunne kjøre prosjektet uten ekstra konfigurasjon. Dette bryter med vanlig best practice, siden en .env ofte inneholder sensitiv informasjon, men vår fil inneholdt kun en felles API-nøkkel som brukes for å hente et stort datasett via vårt fetch-script. Vår .gitignore ekskluderte igjennom selve prosjektet blant annet Python-kompilater og cache (pycache, .pyc), Jupyter-checkpoints, nedlastede dataset-filer (.csv, *.json), virtuelle miljøer, logg- og tmp-filer samt OS-spesifikke filer, men ikke .env filen. 

Rasjonalet bak valget var at datasettet vårt består av flere millioner linjer, og det er ikke praktisk å laste opp disse store filene til repoet. I stedet krever fetch-scriptet at brukeren har en API-nøkkel for å hente dataene dynamisk. Ved å pushe .env tidlig unngikk vi “hardcoded” credentials i koden, samtidig som vi demonstrerte forståelse for environment management og separation of concerns. 
Etter presisering i forelesning har vi fjernet .env-filen i siste commit og sørget for at .gitignore nå ekskluderer den permanent. Dette sikrer at repoet følger sensur-veiledningens krav til filer i versjonskontroll, samtidig som vi opprettholder god dokumentasjon og versjonshåndtering.


## Analyse og visualisering

Vi har valgt å gjøre en grundig statistisk analyse av dataene ved å bruke NumPy og Pandas for å beregne gjennomsnitt, median og standardavvik på både månedlig og årlig nivå. I monthlystats.py og yearlystats.py er funksjoner som compute_monthly_stats og compute_yearly_stats, hvor vi henter ut df.mean(), df.median() og df.std() for å avdekke datasettets sentraltendens og spredning. Denne tilnærmingen har gitt oss innsikt i hvordan temperatur- og nedbørsmønstre varierer over tid, og har vært avgjørende for å vurdere datakvaliteten før videre modellering.

For å undersøke sammenhengen mellom to miljøvariabler har vi inkludert både en kvantitativ korrelasjonsanalyse og visuell fremstilling i de forskjellige notebooksene. Det beregnes korrelasjonskoeffisient og legges deretter en lineær trendlinje på scatterplottet.

For å håndtere skjevheter og ekstreme verdier har vi utviklet en egen modul, outlierdetector.py, med funksjoner som detect_iqr() og remove_outliers_iqr(). Vi bruker IQR-metoden for å identifisere datapunkter utenfor 1.5×IQR og dokumenterer effekten av utligger­rensingen i outliers_visualization.ipynb. Her visualiseres en før-og-etter-statistikk for å vise hvordan fjerning av ekstreme verdier stabiliserer både gjennomsnitt og standardavvik.

Visualisering har vært et sentralt verktøy for å formidle funnene. Vi har brukt histogrammer og boxplots (f.eks. i precipitation_visualization.ipynb) for å kartlegge fordeling og utliggere, linjediagrammer for tidsserier og korrelasjons-heatmaps for å avdekke mønstre på tvers av variabler. Før modellering har vi i tillegg standardisert tallene med StandardScaler fra modulen sklearn.preprocessing, noe som sikrer at alle funksjoner bidrar likt i algoritmer som lineær regresjon.